# RAG加载《围城》的检索

## 基本思路

In [ ]:
技术栈：

- 基于 one-api 选择 llm
- 基于faiss 保存索引，做相似度搜索
- 基于 bge-m3 嵌入模型实现文档切片和向量化

## 准备工作

In [1]:
%%time
%%capture

!pip install langchain
!pip install langchain-openai
!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv

CPU times: user 47.5 ms, sys: 25.6 ms, total: 73.1 ms
Wall time: 14.8 s


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader

%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

In [2]:
system_message=SystemMessage(content="你是小羽，是一个人工智能助手。")

In [3]:
%%time

model_name='/models/bge-m3'
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

CPU times: user 3.48 s, sys: 1.2 s, total: 4.68 s
Wall time: 4.2 s


## 加载文档

In [4]:
%%time
%%capture

documents=[]
loader = TextLoader("./围城.txt")
documents.extend(loader.load())

CPU times: user 2.08 ms, sys: 668 µs, total: 2.75 ms
Wall time: 2.71 ms


## 文档分片

In [9]:
%%time

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=128,chunk_overlap=20)
text_splits=text_splitter.split_documents(documents)

len(text_splits)

CPU times: user 299 ms, sys: 7.92 ms, total: 307 ms
Wall time: 311 ms


2236

In [10]:
%%time

# cpu - Wall time: 19min 9s - 1024
# cuda - Wall time: 1min 31s - 256
# cuda - Wall time: 1min 27s - 128
vectorstore = FAISS.from_documents(text_splits, embeddings)

CPU times: user 1min 26s, sys: 1.23 s, total: 1min 27s
Wall time: 1min 27s


## 搜索

In [12]:
%%time

query = "孙柔嘉"
docs = vectorstore.similarity_search(query,k=10)

for doc in docs:
    display(doc.page_content)

'家小孩子“你的Ｂａｂｙ”，人家太太“你的Ｍｒｓ”那种女留学生。这种姑母，柔嘉当然叫她Ａｕｎｔｉｅ。她年轻时出过风头，到现在不能忘记，对后起的女学生批判甚为严厉。柔嘉最喜欢听她的回忆，所以独蒙怜爱。孙先生夫妇很怕这位姑太太，家里的事大半要请她过问。她丈夫陆先'

'柔嘉自言自语：“她是比你对我好，我家里的人也比你家里的人好。”\n    鸿渐的回答是：“Ｓｈ——ｓｈ——ｓｈ——ｓｈａｗ。”'

'情，仿佛怪他不女起解似的押了柔嘉来。他交From：qili02：40：18－0700'

'他到了孙家两次以后，就看出来柔嘉从前口口声声“爸爸妈妈”，而孙先生孙太太对女儿的事淡漠得等于放任。孙先生是个恶意义的所谓好人——无用之人，在报馆当会计主任，毫无势力。孙太太老来得子，孙家是三代单传，把儿子的抚养作为宗教，打扮得他头光衣挺，像个高等美容院里'

'大到孙家去请她，表示欢迎。这样一来，她可以比较不陌生。”三奶奶沉着脸，二奶奶说：“姐姐，你真是好脾气！孙柔嘉是什么东西，摆臭架子，要我们去迎接她！我才不肯呢。”二奶奶说：“她今天不肯来是不会来了。猜准她快要养了，没有脸到婆家来，今天推明天，明天推后天，咱们'

...

'嘴，不敢太针锋相对。她们把两间房里的器具细看，问了价钱，同声推尊柔嘉能干津明，会买东西，不过时时穿插说：“我在什么地方也看见这样一张桌子（或椅子），价钱好像便宜些，可惜我没有买。”三奶奶问嘉道：“你有没有搁箱子的房间？”柔嘉道：“没有。我的箱子不多，全搁在'

'心。你瞧，这就是大学毕业生！”二奶奶对丈夫发表感想如下：“你留心没有？孙柔嘉脸上一股妖气，一看就是人上邪道女人，所以会干那种无耻的事。你父亲母亲一对老糊涂，倒赞她美！不是我吹牛，我家的姊妹多少正经干净，别说从来没有男朋友，就是订了婚，跟未婚夫通信爹都不许的'

'的心全偏到夹肢窝里的！老大一个人大学毕业留洋，钱花得不少了，现在还要用老头的钱。我就不懂，他留了洋有什么用，别说比不上二哥了，比我们老三都不如。”二奶奶道：“咱们瞧女大学生‘自立’罢。”二人旧嫌尽释，亲爇得有如结义姐妹（因为亲生姐妹倒彼此忌嫉的），孙柔嘉做'

'呢，你吃的亏就是没留过学。我在德国，就知道德国妇女的三Ｋ运动：Ｋｉｒｃｈｅ，Ｋｎｅｃｈｅ，Ｋｉｎｄｅｒ——”柔嘉道：“我不要听，随你去说。不过我今天才知道，你是位孝子，对你父亲的话这样听从——”这吵架没变严重，因为不能到孙家去吵，不能回方家去吵，不宜在路上'

CPU times: user 34.6 ms, sys: 24 ms, total: 58.6 ms
Wall time: 66.3 ms


## 保存索引

In [13]:
%%time

index_path='weicheng_index'
vectorstore.save_local(index_path)

CPU times: user 10.6 ms, sys: 7.86 ms, total: 18.4 ms
Wall time: 19.4 ms


## 加载索引

In [5]:
%%time

index_path='weicheng_index'
vectorstore=FAISS.load_local(index_path,
                             embeddings,
                            allow_dangerous_deserialization=True)
vectorstore.index.ntotal

CPU times: user 133 ms, sys: 5.56 ms, total: 139 ms
Wall time: 141 ms


2236

## 直接llm对话

In [21]:
%%time

chat = ChatOpenAI(api_key=token, 
                  base_url=base_url,
                  model='xiaoyu', 
                  temperature=0,
                  streaming=True)

messages = [
    system_message,
    HumanMessage(content="孙柔嘉的丈夫是谁？"),
]

for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

孙柔嘉的丈夫是陆焉识，这是小说《围城》中的人物。陆焉识是一位留洋归来的学者，与孙柔嘉有过婚姻关系。CPU times: user 188 ms, sys: 9.34 ms, total: 197 ms
Wall time: 1.57 s


## 基于嵌入上下文的llm对话

In [23]:
%%time

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = VectorStoreRetriever(vectorstore=vectorstore,search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=retriever
)

question='孙柔嘉的丈夫是谁？'
qa_chain.invoke({"query": question})

CPU times: user 47.8 ms, sys: 2.58 ms, total: 50.4 ms
Wall time: 507 ms


{'query': '孙柔嘉的丈夫是谁？', 'result': '孙柔嘉的丈夫是陆先生。'}

## 增加嵌入数量可能改善对话结果生成

In [34]:
%%time

retriever = VectorStoreRetriever(vectorstore=vectorstore,search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=retriever
)

question='孙柔嘉的丈夫是谁？'
qa_chain.invoke({"query": question})

CPU times: user 49.6 ms, sys: 20.6 ms, total: 70.2 ms
Wall time: 1.26 s


{'query': '孙柔嘉的丈夫是谁？', 'result': '孙柔嘉的丈夫是方鸿渐。'}

## 改为流式输出 - 但结果有问题

In [42]:
%%time

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(chat, prompt)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context_docs=None

print('回答: \n')

for chunk in retrieval_chain.stream({"input": "孙柔嘉的丈夫是谁？"}):
    if 'answer' in chunk:
        print(chunk['answer'], end="", flush=True)
    elif 'context' in chunk:
        context_docs=chunk['context']
print()

print('\n相似度上下文: \n')
for doc in context_docs:
    print(doc.page_content)
    print()

回答: 

孙柔嘉的丈夫是陆先生。

相似度上下文: 

他到了孙家两次以后，就看出来柔嘉从前口口声声“爸爸妈妈”，而孙先生孙太太对女儿的事淡漠得等于放任。孙先生是个恶意义的所谓好人——无用之人，在报馆当会计主任，毫无势力。孙太太老来得子，孙家是三代单传，把儿子的抚养作为宗教，打扮得他头光衣挺，像个高等美容院里

家小孩子“你的Ｂａｂｙ”，人家太太“你的Ｍｒｓ”那种女留学生。这种姑母，柔嘉当然叫她Ａｕｎｔｉｅ。她年轻时出过风头，到现在不能忘记，对后起的女学生批判甚为严厉。柔嘉最喜欢听她的回忆，所以独蒙怜爱。孙先生夫妇很怕这位姑太太，家里的事大半要请她过问。她丈夫陆先

太太，家里的事大半要请她过问。她丈夫陆先生，一脸不可饶恕的得意之色，好谈论时事。因为他两耳微聋，人家没气力跟他辩，他心里只听到自己说话的声音，愈加不可理喻。夫妇俩同在一家大纱厂里任要职，先生是总工程师，太太是人事科科长。所以柔嘉也在人事科里找到位置。姑太太

心。你瞧，这就是大学毕业生！”二奶奶对丈夫发表感想如下：“你留心没有？孙柔嘉脸上一股妖气，一看就是人上邪道女人，所以会干那种无耻的事。你父亲母亲一对老糊涂，倒赞她美！不是我吹牛，我家的姊妹多少正经干净，别说从来没有男朋友，就是订了婚，跟未婚夫通信爹都不许的

孙柔嘉在订婚以前，常来看鸿渐；订了婚，只有鸿渐去看她，她轻易不肯来。鸿渐最初以为她只是个女孩子，事事要请教自己；订婚以后，他渐渐发现她不但很有主见，而且主见很牢固。她听他说准备退还聘约，不以为然，说找事不容易，除非他另有打算，别逞一时的意气。鸿渐问道：“

CPU times: user 104 ms, sys: 24 ms, total: 128 ms
Wall time: 603 ms


## 增大k值有时会有效果

In [44]:
%%time

retriever = vectorstore.as_retriever(search_kwargs={"k": 15})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context_docs=None

print('回答: \n')

for chunk in retrieval_chain.stream({"input": "孙柔嘉的丈夫是谁？"}):
    if 'answer' in chunk:
        print(chunk['answer'], end="", flush=True)
    elif 'context' in chunk:
        context_docs=chunk['context']
print()

回答: 

孙柔嘉的丈夫是方鸿渐。
CPU times: user 98.3 ms, sys: 20.5 ms, total: 119 ms
Wall time: 549 ms


## 但并不稳定，完全碰运气

In [48]:
%%time

retriever = vectorstore.as_retriever(search_kwargs={"k": 200})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context_docs=None

print('回答: \n')

for chunk in retrieval_chain.stream({"input": "孙柔嘉的丈夫是谁？"}):
    if 'answer' in chunk:
        print(chunk['answer'], end="", flush=True)
    elif 'context' in chunk:
        context_docs=chunk['context']
print()

回答: 

徐志摩
CPU times: user 83.5 ms, sys: 19.1 ms, total: 103 ms
Wall time: 410 ms


## 使用rerank